In [1]:
import os
# import plotly.graph_objects as go
from tqdm import tqdm
from gym import envs
import argparse
import numpy as np
from bigmdp.data.buffer import StandardBuffer,ReplayBuffer, gather_data_in_buffer, get_iter_indexes
from bigmdp.mdp.MDP_GPU import FullMDP
from bigmdp.utils.utils_eval import evaluate_on_env
from bigmdp.mdp.agent import SimpleAgent
from copy import deepcopy as cpy
from os import path
from arg_def import * 
import gym
from sklearn.neighbors import KDTree
from IPython import display
import torch
import time
import copy

In [7]:
parser, ArgumentDict = get_argument_parser()
options = "--env_name CartPole-v0 --MAX_S_COUNT 15000 --MAX_NS_COUNT 5 --mdp_build_k 5 --policy_k 11 --normalize_by_distance"
args = parser.parse_args(options.split(" "))

for title, arg_names in ArgumentDict.items():
    print_args(args, to_show_args=arg_names, title = title)

##################################################    envArgs    ##################################################
env_name                       : CartPole-v0                    seed                           : 4444
###################################################################################################################
##################################################    mdpBuildArgs    ##################################################
unknown_transition_reward      : -1000                          rmax_reward                    : 10000
balanced_exploration           : 0                              rmax_threshold                 : 2
MAX_S_COUNT                    : 15000                          def_device                     : GPU
fill_with                      : 0Q_src-KNN                     mdp_build_k                    : 5
knn_delta                      : 1e-08                          penalty_type                   : linear
penalty_beta                   : 1     

In [ ]:
from image_wrappers import *
env = gym.make(args.env_name)

In [ ]:
train_buffer = ReplayBuffer(state_dim = env.observation_space.shape[0],
                           is_atari= False, 
                           atari_preprocessing= None, 
                           batch_size=32, 
                           buffer_size=20000,
                           device="cpu")

In [ ]:
train_buffer, info = gather_data_in_buffer(train_buffer, env,policy = lambda s:np.random.randint(2), episode_count=99999, frame_count=10000)

In [ ]:
class DummyNet():
    def __init__(self, sim, add_noise=False):
        self.simulator = sim

    def encode_single(self, o):
        return tuple(o)

    def encode_batch(self, o_batch):
        return [tuple(o) for o in o_batch]

    def predict_single_transition(self, o, a):
        assert False, "Not Implemented Error"

    def predict_batch_transition(self, o_batch, a_batch):
        assert False, "Not Implemented Error"

In [7]:
empty_MDP = FullMDP(A= list(range(env.action_space.n)),
                    ur=args.unknown_transition_reward,
                    vi_params={"gamma":  args.gamma,
                               "slip_prob": args.slip_probability,
                               "rmax_reward": args.rmax_reward,
                               "rmax_thres": args.rmax_threshold,
                               "balanced_explr": args.balanced_exploration,
                              "rmin":-1000},
                    knn_delta=args.knn_delta,
                    MAX_S_COUNT=args.MAX_S_COUNT,
                    MAX_NS_COUNT=args.MAX_NS_COUNT,
                    default_mode=args.def_device)

myAgent =  SimpleAgent(mdp_T= empty_MDP, net = DummyNet(None), fill_with = args.fill_with,
                       mdp_build_k = args.mdp_build_k, plcy_k = args.policy_k[0],  
                       kNN_on_sa = args.smooth_with_seen, soft_at_plcy = args.soft_q, 
                       normalize_by_distance= args.normalize_by_distance,
                       penalty_type=args.penalty_type, penalty_beta = args.penalty_beta,abstraction_flag=False)

myAgent.build_mdp(train_buffer, verbose= True)

 14%|█▍        | 1424/10012 [00:00<00:00, 14237.23it/s]

Step 1 (Parse Transitions):  Running
Step 1 [Parse Transitions]:  Complete,  Time Elapsed: 0.054155588150024414


Step 2 [Seed Seen Transitions + Unknown (s,a) pairs]:  Running
Len of to commit transitions 10012
ABstraction Faldg False


  2%|▏         | 395/20026 [00:00<00:04, 3946.27it/s]

Len of to commit sa pairs 20026
Step 2 (Commit Seen Transitions):  Complete,  Time Elapsed: 0.6531634330749512 


Step 3 [Commit all Transitions]:  Running


100%|██████████| 20026/20026 [00:04<00:00, 4020.24it/s]


Step 3 (Commit UnSeen Transitions):  Complete,  Time Elapsed: 4.983672380447388
Step 4 [Solve MDP]:  Running
Elapsed Time:0s, VI Error:0.08272552, #Backups: 250
Elapsed Time:1s, VI Error:0.00673676, #Backups: 500
Elapsed Time:1s, VI Error:0.00058746, #Backups: 750
Time takedn to solve 1.9754598140716553
% of missing trans 0.0
Step 4 [Solve MDP]:  Complete,  Time Elapsed: 3.0332117080688477




In [8]:
print(f"Policy Parmeters - policy_k:{myAgent.plcy_k}, Weighted Neighbors:{myAgent.norm_by_dist}, kNN_on_sa:{myAgent.kNN_on_sa}, soft:{myAgent.soft_at_plcy},")
sum_reward_running = {policy_name:evaluate_on_env(env,  
                                                  policy, 
                                                  eps_count=50,progress_bar=True)[0]
                      for policy_name,policy in myAgent.policies.items()}
print(sum_reward_running)

  4%|▍         | 2/50 [00:00<00:03, 13.59it/s]

Policy Parmeters - policy_k:11, Weighted Neighbors:True, kNN_on_sa:False, soft:False,


100%|██████████| 50/50 [00:03<00:00, 15.76it/s]

{'optimal': 200.0, 'random': 20.16, 'eps_optimal': 200.0, 'safe': 200.0}
